In [ ]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm.keras import TqdmCallback

import pandas as pd
import numpy as np
import seaborn as sns
from pandas.core.indexing import _IndexSlice
import matplotlib.pyplot as plt
from functools import partial
import umap
from sklearn.preprocessing import StandardScaler
from pandas.core.dtypes.common import is_numeric_dtype, is_object_dtype
import datetime

from pathlib import Path

import common_functions as fnc

idx: _IndexSlice = pd.IndexSlice

In [ ]:
from data_import import df, samples, didx, DATA_PATH

## Example plots

In [ ]:
# Example plots
ax = df.loc[:, didx(
    fluorometer="MULTI-COLOR-PAM",
    CO2_level="Air", 
    strain="Chlorella vulgaris",
    # SP_color=455
)].dropna().plot(legend=False)
ax.set_xscale("log")
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Fluorescence [Detector V]")
ax.set_title("MCPAM - Example")

ax = df.loc[:, didx(fluorometer="AquaPen", CO2_level="Air")].dropna().plot(legend=False)
ax.set_xscale("log")
ax.set_xlabel("Time [ms]")
ax.set_ylabel("Fluorescence [AU]")
ax.set_title("AquaPen - Example")

# Model training

## Select the data to be trained on

In [4]:
dat = df.loc[
    0.01:, : # Exclude data before the light pulse
    # didx(
    #     fluorometer="MULTI-COLOR-PAM", # Only use MCPAM data
    #     strain='Synechocystis sp. PCC 6803', # Only use Synechocystis data
    # )
].dropna()

### Map the treatment effects

In [5]:
test  = pd.read_csv(
    DATA_PATH / "effects_map.csv",
    header=[0,1],
    index_col=[0,1],
    ).astype(float).fillna(0).astype(bool)

In [6]:
# Read the map of effects transformed into one-hot encoding
effects_map_raw = pd.read_csv(
    DATA_PATH / "effects_map.csv",
    header=[0,1],
    index_col=[0,1],
    ).astype(float).fillna(0).astype(bool)

# Exclude Light intensity and temperature from targets
effects_map = effects_map_raw[[
    'control_measurement',
    'PSII_closed',
    'CBB_inhibited',
    'TOX_inhibited',
    'electron_drain'
]]

# Set High light as a closing factor of PSII
effects_map.loc[:,'PSII_closed'] = np.logical_or(
    effects_map['PSII_closed'].to_numpy(),
    effects_map_raw["high_light"].to_numpy()
)

# Ass high and low temperature as 
effects_map.loc[:,'control_measurement'] = np.logical_or(
    effects_map['control_measurement'].to_numpy(),
    effects_map_raw["low_temperature"].to_numpy()
)
effects_map.loc[:,'control_measurement'] = np.logical_or(
    effects_map['control_measurement'].to_numpy(),
    effects_map_raw["high_temperature"].to_numpy()
)

# Get the effects and map the mto the targets
effects = samples.loc[dat.columns.get_level_values(0), ["Effect in PSET", "Treatment"]]

targets = effects_map.loc[pd.MultiIndex.from_frame(effects)].droplevel(1, axis=1)
targets = targets.astype(int)
targets.index = dat.columns

# Make a Multiindex with a duplicated entries
target_names = targets.columns
targets.columns = pd.MultiIndex.from_arrays([targets.columns, targets.columns])

# Select features

**Message from Tomas:**

INPUTS
- Strain: Synechocystis / Chlorella. Later, when we have more strains measured, we can make it more general like green algae/cyanobacteria.
- Temperature: **it could be enough to keep it in intervals, like e.g. 15-20 / 20-25 / 25-30 / 30-35 etc.**
- Light acclimation state: light-acclimated / dark-acclimated
- OD: needs to be standardized = measured by one device since each device measure OD differently.  In the dataset we now have it for Multi-Cultivator, but it would be better to refer to UV-Vis and 1 cm cuvette. Alternatively, chlorophyll content could work well. I can provide all these values, based on additional measurements that we have already performed.
- SP color: **blue / orange-red could be enough**
- SP Intensity: saturating / non-saturating. This parameter definitely matters, but it could be tricky for the users to validate if SP is saturating or not. So, alternatively, the user can provide SP intensity (**perhaps again in some intervals like < 1000 / 1000-1500 / 1500-2000 / 2000-2500 / >2500**) and we can evaluate by our algorithm if the SP was saturating, based on the selection of strain, OD, SP color and SP intensity.
- Fluorometer: MCPAM / AquaPen - I would keep it for now, since I am still not sure if there is any fundamental difference between MC-PAM, AquaPen and FL-6000, or not. But in general, it should be only SP color and SP intensity that matter the most
- These parameters might be not necessary, but I am not sure - the OJIP curves were different under these specific acclimations, so it could make the predictions better if we discriminate also these acclimations - but on the other hand, it might be overkill:
  - CO2 level
  - Growth light color

OUTPUTS
- PSII closed: DCMU, high light
- CBB inhibited: Glycolaldehyde
- TOX inhibited: KCN
- Electron drain from PSI: Methylviologen. Btw. yes, this effect should be the opposite of Glycolaldehyde treatment
- **PQ redox state: reduced / oxidized**
- It would be great to include more inhibitors, to describe more effects -we can do that later:
  - Cyt b6/f closed: DBMIB
  - Cyclic electron flow around PSI closed: Antimycin A3


## Get experimental conditions

In [7]:
# Get the conditions as the Multiindex-columns
conditions = dat.columns.to_frame()
# conditions.index = dat.columns.get_level_values(0)

# Select the relevant columns
condition_types = pd.Series({
    'Strain': "string",
    'CO2 level': "numeric", # There is a meaning to a higher CO2 concentration (maybe make categorical?)
    'Cultivation + experiment temperature': "numeric",
    'Cultivation light intensity': "numeric",
    'Dark or light acclimated': "string",
    'Growth light color (nm)': "string",
    'Fluorometer': "string",
    'SP color (nm)': "string", # There is no linear relationship between wavelength and effect
    'SP intensity': "numeric",
    'OD680 MC-1000': "numeric",
    'OD720 MC-1000': "numeric",
})

conditions = conditions[condition_types.index]

# Replace certain column values

# Replace CO2 level with the actual (assumed) numerical ppm
conditions["CO2 level"] = conditions["CO2 level"].replace({
    "Air": "0.0004",
    "High CO2": "0.05"
}).astype(float)

# Replace SP color with categorical value because the numerical gradient is not meaningful
conditions["SP color (nm)"] = conditions["SP color (nm)"].astype(str)

# Make a Multiindex with a duplicated entries
conditions.columns = pd.MultiIndex.from_arrays([conditions.columns, conditions.columns])

# Encode conditions in one-hot
categorical_conditions = condition_types[condition_types == "string"].index.to_numpy()
numerical_conditions = condition_types[condition_types != "string"].index.to_numpy()

## Sample OJIP

In [ ]:
# Select the number of sampled points
n_points = 40

# Time points, logspaced
log_time_points = np.linspace(
    np.log10(dat.index[0]),
    np.log10(dat.index[-1]),
    n_points
)
time_points = 10 ** log_time_points

# Pre-populate the interp function
_interp = partial(np.interp, time_points, dat.index)

# Interpolate the selected points
ojip_sampled = dat.apply(_interp)
ojip_sampled.index = pd.MultiIndex.from_product([
    ["ojip"],
    ["ojip_" + x for x in log_time_points.round(2).astype(str)]
])

# Add sampled points to features
ojip_sampled = ojip_sampled.T

# Subset the data to the samples and time to be included in the analysis 
ax = dat.plot(legend=False)

for t in time_points:
    ax.axvline(t, c="k", alpha=0.5)

ax.set_xscale("log")

# Add data types
ojip_types = pd.Series({"ojip":"time-series-gradients"})

## Collect and split dataset

In [ ]:
# Collect all data sets
dat_sets = [
    conditions,
    ojip_sampled,
    targets
]
dat_full = pd.concat(dat_sets, axis=1)

# Split data into training set
dat_train, _dat_trainval = train_test_split(
    dat_full,
    test_size=0.2, 
    random_state=42,
    stratify=dat_full[target_names]
)

# Split data into test and validation set
dat_test, dat_val = train_test_split(
    _dat_trainval,
    test_size=0.5, 
    random_state=42,
    stratify=_dat_trainval[target_names]
)

print(f"Dimensions train: {dat_train.shape}, test: {dat_test.shape}, val: {dat_val.shape}")


if not np.all(dat_train[target_names].drop_duplicates().sum(axis=0) == 1):
    raise RuntimeError("Not all targets are in the training set")

# Make into dataset
train_ds = fnc.df_to_dataset(dat_train, targets=target_names, batch_size=64, shuffle=True)
test_ds = fnc.df_to_dataset(dat_test, targets=target_names, shuffle=False)
val_ds = fnc.df_to_dataset(dat_val, targets=target_names, shuffle=False)

## Prepare data

In [ ]:
# Get the types of all features
feature_types = pd.concat([
    condition_types,
    ojip_types
])

# Create containers for inputs and encodings
all_inputs = {}
encoded_features_dict = {}
encoded_features = []

# Encode all features
for col_name, col_dtype in feature_types.items():
    # Create a numeric normalisation layer
    if col_dtype == "numeric":
        col = layers.Input(shape=(1,), name=col_name)
        normalization_layer = fnc.get_normalization_layer(col_name, train_ds)
        encoded_col = normalization_layer(col)
    
    # Create a string enconding layer, could also work for integer encoding
    elif col_dtype == "string":
        col = layers.Input(shape=(1,), name=col_name, dtype='string')
        encoding_layer = fnc.get_category_encoding_layer(name=col_name,
                                                    dataset=train_ds,
                                                    dtype='string',
                                                    max_tokens=5)
        encoded_col = encoding_layer(col)

    # Create a layer to normalise time series and calculate gradients
    elif col_dtype == "time-series-gradients":
        col = layers.Input(shape=(dat_full[col_name].shape[1],), name=col_name)
        reshaped_col = layers.Reshape((dat_full[col_name].shape[1], 1))(col)
        normalization_layer = fnc.NormalizedTimeSeriesWithDerivatives()
        encoded_col = normalization_layer(reshaped_col)
        
    else:
        raise KeyError(f"No handling for col_dtype {col_dtype} defined")

    all_inputs[col_name] = col
    encoded_features.append(encoded_col)
    encoded_features_dict[col_name] = encoded_col

# Define the preprocessing layer as a model
preprocessing_layer = keras.Model(
    all_inputs,
    encoded_features_dict,
    name="preprocessing_layer"
)

# Plot the preprocessing layer
keras.utils.plot_model(
    preprocessing_layer,
    show_shapes=True,
    show_layer_names=True,
    # rankdir="LR",
    to_file="figures/test_preprocessing.png",
    dpi=100
)

## Create UMAP mapping

In [ ]:
# Concatenate the outputs of the preprocessing layer to perform UMAP
preprocessed = preprocessing_layer(all_inputs)

# Append the derivatives of the ojip signal to the end to create a long feature vector
preprocessed["ojip"] = layers.Flatten()(preprocessed["ojip"])

concatenated_preprocess = keras.Model(
    all_inputs,
    layers.concatenate(list(preprocessed.values())),
    name="concatenation_layer"
)

# Set a random seed for UMAP
UMAP_seed = 2025

# Scale the features
df_features_scaled = concatenated_preprocess.predict(
    fnc.get_dataset_from_input_df(dat_full, all_inputs)
)

# Create the UMAP embedding
reducer = umap.UMAP(random_state=UMAP_seed)
embedding = pd.DataFrame(
    reducer.fit_transform(df_features_scaled),
    index=dat_full.index,
    columns=["UMAP_1", "UMAP_2"]
).reset_index()

# Plot
categories = df.columns.names[1:]
fig, axes = plt.subplots(
    int(np.ceil(len(categories)/3)),
    3,
    figsize=(7,15),
    sharey=True,
    sharex=True,
)

for category, ax in zip(categories, axes.flatten()):
    sns.scatterplot(
        embedding,
        x="UMAP_1",
        y="UMAP_2",
        hue=category,
        ax=ax,
        legend=False
    )
    ax.set_title(category)

    if len(embedding[category].value_counts()) == 1:
        ax.text(s="one category",x=0.98, y=0.98, ha="right", va="top", transform=ax.transAxes, size=7)

fig.tight_layout()
fig.savefig("figures/umap_conditions.png")

In [ ]:
# Plot targest on UMAP
# Add UMAP to targets
embedding_targets = pd.concat([
    targets.droplevel(-1, axis=1).droplevel(list(range(1,20)), axis=0),
    embedding.set_index("Label").loc[:, ["UMAP_1", "UMAP_2"]],
], axis=1)

# Plot
categories = effects_map.columns.get_level_values(0)
fig, axes = plt.subplots(
    int(np.ceil(len(categories)/3)),
    3,
    figsize=(7,5),
    sharex=True,
    sharey=True
)

for category, ax in zip(categories, axes.flatten()):
    sns.scatterplot(
        embedding_targets,
        x="UMAP_1",
        y="UMAP_2",
        hue=category,
        ax=ax,
        legend=False
    )
    ax.set_title(category)

    if len(embedding_targets[category].value_counts()) == 1:
        ax.text(s="one category",x=0.98, y=0.98, ha="right", va="top", transform=ax.transAxes, size=7)

fig.tight_layout()
fig.savefig("figures/umap_conditions.png")

## Model functions

In [13]:
def split_output_to_dict(dense_output):
    outputs = {
        "control_measurement": layers.Lambda(lambda x: tf.expand_dims(x[:, 0], axis=-1), name="control_measurement")(dense_output),
        "PSII_closed": layers.Lambda(lambda x: tf.expand_dims(x[:, 1], axis=-1), name="PSII_closed")(dense_output),
        "CBB_inhibited": layers.Lambda(lambda x: tf.expand_dims(x[:, 2], axis=-1), name="CBB_inhibited")(dense_output),
        "TOX_inhibited": layers.Lambda(lambda x: tf.expand_dims(x[:, 3], axis=-1), name="TOX_inhibited")(dense_output),
        "electron_drain": layers.Lambda(lambda x: tf.expand_dims(x[:, 4], axis=-1), name="electron_drain")(dense_output),
    }
    return outputs

## Create test model

In [ ]:
## Test model
# Get the preprocessed inputs
preprocessed = preprocessing_layer(all_inputs)

# Append the derivatives of the ojip signal to the end to create a long feature vector
preprocessed["ojip"] = layers.Flatten()(preprocessed["ojip"])

# Concatenate all features
x = layers.concatenate(list(preprocessed.values()))

# Dense hidden layer
x = layers.Dense(32, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# Dense layer for output calculation
# Uses sigmoid activation function for output between 0 and 1
dense_output = layers.Dense(targets.shape[1], activation="sigmoid")(x)

# Split the output into a dictionary
outputs = split_output_to_dict(dense_output)


# Create and compile test model
test_model = keras.Model(all_inputs, outputs)
test_model.compile(
    loss='binary_crossentropy', # Expects output to be multiple probabilities for classes
    optimizer='adam',
    metrics={key:[ # Calculate metrics for each target
        keras.metrics.BinaryAccuracy(threshold=0.5, name="..binary_accuracy"), # Accuracy (assumes threshold 0.5)
        keras.metrics.Recall(thresholds=0.5, name="..recall"), # Recall (assumes threshold 0.5)
        keras.metrics.Precision(thresholds=0.5, name="..precision"), # Precision (assumes threshold 0.5)
        keras.metrics.F1Score(threshold=0.5, name="..f1_score"), # F1-score (assumes threshold 0.5)
        ] for key in target_names},
)

# Plot the model
keras.utils.plot_model(
    test_model,
    show_shapes=True,
    show_layer_names=True,
    to_file="figures/test_model.png",
    dpi=100
)

In [ ]:
# Train the model

# Use tensorboard to track the training
use_tensorboard = False
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = [keras.callbacks.TensorBoard(log_dir=log_dir)] if use_tensorboard else []

test_model_history = test_model.fit(
    train_ds,
    epochs=300,
    batch_size=32,
    verbose=0,
    validation_data=val_ds,
    callbacks=[
        TqdmCallback(verbose=1)
    ] + tensorboard_callback
)

# Plot the loss over the Epochs
fnc.plot_loss_development(test_model_history)

### Evaluate on the test data

In [ ]:
# Evaluate the model on the test data
test_model_eval = test_model.evaluate(test_ds, return_dict=True)

fnc.plot_model_metrics(test_model_eval, test_ds)

In [ ]:
# Exemplary prediction
fnc.predict_model_from_df(test_model, dat_test).head()

## Create machine learning model

In [ ]:
## Define the model
# Get the preprocessed inputs
seperated_inputs = preprocessing_layer(all_inputs)

# Add an LSTM layer to the OJIP input
seperated_inputs["ojip"] = layers.LSTM(16, activation="tanh", name="LSTM_ojip")(seperated_inputs["ojip"])
seperated_inputs["ojip"] = layers.Dropout(0.5, name="LSTM_1_Dropout")(seperated_inputs["ojip"])

# Concatenate all features
x = layers.concatenate(list(seperated_inputs.values()))

# Dense hidden layer
x = layers.Dense(16, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# Dense layer for output calculation
# Uses sigmoid activation function for output between 0 and 1
dense_output = layers.Dense(targets.shape[1], activation="sigmoid")(x)

# Split the output into a dictionary
outputs = split_output_to_dict(dense_output)

# Create and compile test model
model = keras.Model(all_inputs, outputs)
model.compile(
    loss='binary_crossentropy', # Expects output to be multiple probabilities for classes
    optimizer='adam',
    metrics={key:[ # Calculate metrics for each target
        keras.metrics.BinaryAccuracy(threshold=0.5, name="..binary_accuracy"), # Accuracy (assumes threshold 0.5)
        keras.metrics.Recall(thresholds=0.5, name="..recall"), # Recall (assumes threshold 0.5)
        keras.metrics.Precision(thresholds=0.5, name="..precision"), # Precision (assumes threshold 0.5)
        keras.metrics.F1Score(threshold=0.5, name="..f1_score"), # F1-score (assumes threshold 0.5)
        ] for key in target_names},
)

# Plot the model
keras.utils.plot_model(
    model,
    show_shapes=True,
    show_layer_names=True,
    to_file="figures/model.png",
    dpi=100
)

In [ ]:
# Train the model

# Use tensorboard to track the training
use_tensorboard = False
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = [keras.callbacks.TensorBoard(log_dir=log_dir)] if use_tensorboard else []

model_history = model.fit(
    train_ds,
    epochs=800,
    verbose=0,
    validation_data=val_ds,
    callbacks=[
        TqdmCallback(verbose=1)
    ] + tensorboard_callback
)

# Plot the loss over the Epochs
fnc.plot_loss_development(model_history)

In [ ]:
len(list(train_ds.map(lambda x,y: y).as_numpy_iterator()))

In [ ]:
# Plot the final metrics of a model
def plot_history_metrics(history, fit_dataset, val_dataset):
    # Get the targets from the dataset to count occurrences
    target_numbers={}
    for nam, ds in {"fit":fit_dataset, "val":val_dataset}.items():
        _targets_subsets = list(ds.map(lambda x,y: y).as_numpy_iterator())
        _targets = [pd.DataFrame({k:v.flatten() for k,v in _targets_subset.items()}) for _targets_subset in _targets_subsets]
        _targets = pd.concat(_targets, axis=0)

        target_numbers[nam] = _targets.sum().sort_values(ascending=False)

    metrics = history.history

    # Get all metrics and assign them to the targets
    plot_metrics = pd.DataFrame(
        {tuple(k.split("_..")): v for k,v in metrics.items() if not k.endswith("loss")}
    )

    print(plot_metrics.columns.levels[0])

    metrics_names = plot_metrics.columns.levels[1]
    targets_names = _targets.columns

    # Create the figure
    fig, axes = plt.subplots(
        len(targets_names), 
        2,
        figsize=(7,7),
        sharex=True
    )

    # Plot each target
    for j, val in enumerate(["fit", "val"]):
        for i, _target in enumerate(targets_names):
            prefix = "val_" if val=="val" else ""
            target = prefix + _target
            axes[i, j].plot(
                plot_metrics.loc[:, idx[target, metrics_names]],
                label=metrics_names
            )
            axes[i, j].set_title(f"{target} (n={target_numbers[val][_target]})")
            axes[i, j].set_ylabel(target)

        axes[-1, j].set_xlabel("Epoch No.")

    fig.tight_layout()

    axes[0, -1].legend(loc="upper left", bbox_to_anchor=(1,1))

    return fig, ax

In [ ]:
plot_history_metrics(history, train_ds, val_dataset=val_ds)

In [ ]:
dat_train.shape

In [ ]:
# Evaluate the model on the test data
model_eval = model.evaluate(test_ds, return_dict=True)

fnc.plot_model_metrics(model_eval, test_ds)

# Look at all models

In [ ]:
fig, axes = plt.subplots(len(models_metrics), sharex=True)

for model, ax in zip(models_metrics, axes.flatten()):
    # Plot the model metrics
    models_metrics[model].plot(kind="bar", ax=ax)
    ax.set_title(model)

In [ ]:

# model.compile(loss='mae', optimizer='adam')
# model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=0)

# print(model.evaluate(X_test, y_test))
# # 10.704551696777344

# # normalize the inputs outside the model
# normalizer = Normalization()
# normalizer.adapt(X_train)

# X_train_normalized = normalizer(X_train)
# X_test_normalized = normalizer(X_test)

# inputs = Input(shape=[None, 1])
# x = LSTM(4, return_sequences=True)(inputs)
# x = LSTM(2, return_sequences=True)(x)
# x = LSTM(2, return_sequences=True)(x)
# x = LSTM(4, return_sequences=True)(x)
# x = TimeDistributed((Dense(1)))(x)
# model = Model(inputs, x)

# model.compile(loss='mae', optimizer='adam')
# model.fit(X_train_normalized, y_train, batch_size=32, epochs=10, verbose=0)

# print(model.evaluate(X_test_normalized, y_test))
# # 10.748750686645508

In [64]:
# import tensorflow as tf

# class TimeSeriesNormalization(layers.Layer):
#     def __init__(self, epsilon=1e-6):
#         super(TimeSeriesNormalization, self).__init__()
#         self.epsilon = epsilon  # To prevent division by zero

#     def call(self, inputs):
#         """
#         Normalize each time series independently to zero mean and unit variance.

#         Args:
#             inputs: Tensor of shape (batch_size, time_steps, features)

#         Returns:
#             Normalized tensor of the same shape
#         """
#         mean = tf.reduce_mean(inputs, axis=1, keepdims=True)  # Compute mean along time axis
#         std = tf.math.reduce_std(inputs, axis=1, keepdims=True)  # Compute std along time axis

#         return (inputs - mean) / (std + self.epsilon)  # Normalize

# # Example usage
# batch_size, time_steps, features = 32, 100, 5
# input_data = tf.random.normal((batch_size, time_steps, features))  # Simulated time series data

# normalization_layer = TimeSeriesNormalization()
# normalized_data = normalization_layer(input_data)

# print("Input shape:", input_data.shape)
# print("Normalized shape:", normalized_data.shape)


In [78]:
# fig,ax = plt.subplots()
# ax.plot(normalized_data.numpy().std(axis=1))
# ax.set_ylim(-1,3)